In [146]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers

In [147]:
# Define the neural network architecture
def create_model(input_shape):
    model = models.Sequential()
    
    model.add(layers.Input(shape=input_shape))
    
    # Encoder layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    
    # Bottleneck layer
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    
    # Decoder layers
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))  # Output with 3 channels for RGB images
    
    return model

In [148]:
# Create the model
input_shape = (512, 512, 3)
model = create_model(input_shape)


In [149]:
# Compile the model
model.compile(optimizer='adam', loss=losses.mean_squared_error)

In [162]:
# Define dataset paths
original_images_path = "data/style1/original_images/"
stylized_images_path = "data/style1/stylized_images/"

In [163]:
# Create a dataset
original_dataset = tf.data.Dataset.list_files(original_images_path + '*.jpg')
stylized_dataset = tf.data.Dataset.list_files(stylized_images_path + '*.jpg')
dataset = tf.data.Dataset.zip((original_dataset, stylized_dataset))

In [165]:
# Image preprocessing function
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (512, 512))  # Resize to match the model's input size
    image = tf.cast(image, tf.float32) / 255.0
    return image


In [166]:
# Load and preprocess dataset
dataset = dataset.map(lambda orig, style: (preprocess_image(orig), preprocess_image(style)))

In [167]:
for og, sty in dataset.take(1):
    print(og.shape)

(512, 512, 3)


In [168]:
batch_size = 32
epochs = 10

In [169]:
model.fit(dataset.batch(batch_size),epochs=epochs)

Epoch 1/10


2023-08-18 09:53:55.271132: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.02GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-18 09:53:55.271291: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2023-08-18 09:53:55.306266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-18 09:53:55.654679: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 880.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were a

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_2/conv2d_35/Relu' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/home/gaurav/.local/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/gaurav/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/gaurav/.local/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/gaurav/.local/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib64/python3.11/asyncio/base_events.py", line 607, in run_forever
      self._run_once()
    File "/usr/lib64/python3.11/asyncio/base_events.py", line 1922, in _run_once
      handle._run()
 